In [ ]:
!pip install safetensors gdown pandas transformers sentence-transformers scikit-learn

In [ ]:
pip install pydrive


In [ ]:
!pip install safetensors


In [ ]:
import shutil
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report

#Langkah 1: Pengenalan dan Persiapan Dataset
Kode untuk mengunduh dataset dari Google Drive dan mempersiapkannya:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Tentukan path file yang ingin diunduh
file_path = '/content/drive/MyDrive/UAS TKI/Data/putusan_ma_PNTangerang.csv'

# Tentukan nama file tujuan
file_data = 'putusan_ma_PNTangerang.csv'

# Salin file dari Google Drive ke Colab
shutil.copy(file_path, file_data)

print(f"File berhasil disalin ke: {file_data}")

File berhasil disalin ke: putusan_ma_PNTangerang.csv


In [ ]:
# Load dataset hasil crawling
df = pd.read_csv(file_data, delimiter=',')

# Contoh struktur data
df

,Nomor,Lembaga Peradilan,Barang Bukti,Catatan Amar
0,1556/Pid.Sus/2024/PN Tng,PN TANGERANG,1 (satu) paket plastik klip bening yang berisi...,Menyatakan Terdakwa ADRIAN JUNIARDI als JIGEL ...
1,957/Pid.Sus/2024/PN Tng,PN TANGERANG,1 (satu) buah handphone merk Iphone warna hita...,Menyatakan Terdakwa RAFLY MAULANA ALIAS CEPOT ...
2,946/Pid.Sus/2024/PN Tng,PN TANGERANG,NaN,MENETAPKAN: Menyatakan secara hukum Terdakwa S...
3,838/Pid.Sus/2024/PN Tng,PN TANGERANG,1 (satu) buah plastic klip bening berisikan di...,1. Menyatakan Terdakwa SEPTIAN ADHITIA SULAIMA...
4,1604/Pid.Sus/2024/PN Tng,PN TANGERANG,1 (satu) bungkus plastik klip bening berisikan...,Menyatakan Terdakwa MINDO POGUANTO Anak dari P...
...,...,...,...,...
330,Pdt G PA Mn,PA MADIUN,NaN,Menyatakan Tergugat yang telah dipanggil secar...
331,Pdt G PA Mn,PA MADIUN,NaN,Menyatakan Tergugat yang telah dipanggil secar...
332,Pdt G PA Mn,PA MADIUN,NaN,Menyatakan Tergugat yang telah dipanggil secar...
333,Pdt G PA Mn,PA MADIUN,NaN,Menyatakan Tergugat yang telah dipanggil secar...


In [ ]:
print(df.shape)  # Menampilkan jumlah baris dan kolom


(335, 4)


In [ ]:
print(df['Lembaga Peradilan'].unique())  # Menampilkan nilai unik di kolom 'Lembaga Peradilan'


['PN TANGERANG' 'PA PALU' 'PA BOGOR' 'PN LUBUK LINGAU' 'PT MAKASSAR'
 'PA BENGKALIS' 'PA SERANG' 'PA MADIUN']


In [ ]:
df.dropna(axis=0, inplace=True)

# Menghapus semua baris yang bukan 'PN Tangerang'
df = df[df['Lembaga Peradilan'] != 'PN Tangerang']


In [ ]:
# Bagi dataset menjadi dataset-model dan dataset-evaluasi
dataset_model = df.head(70)
dataset_evaluasi = df.iloc[70:]
dataset_model.to_csv("dataset_model.csv", index=False)
dataset_evaluasi.to_csv("dataset_evaluasi.csv", index=False)

print("Dataset-model dan dataset-evaluasi berhasil disimpan.")

Dataset-model dan dataset-evaluasi berhasil disimpan.


In [ ]:
dataset_model

,Nomor,Lembaga Peradilan,Barang Bukti,Catatan Amar
0,1556/Pid.Sus/2024/PN Tng,PN TANGERANG,1 (satu) paket plastik klip bening yang berisi...,Menyatakan Terdakwa ADRIAN JUNIARDI als JIGEL ...
1,957/Pid.Sus/2024/PN Tng,PN TANGERANG,1 (satu) buah handphone merk Iphone warna hita...,Menyatakan Terdakwa RAFLY MAULANA ALIAS CEPOT ...
3,838/Pid.Sus/2024/PN Tng,PN TANGERANG,1 (satu) buah plastic klip bening berisikan di...,1. Menyatakan Terdakwa SEPTIAN ADHITIA SULAIMA...
4,1604/Pid.Sus/2024/PN Tng,PN TANGERANG,1 (satu) bungkus plastik klip bening berisikan...,Menyatakan Terdakwa MINDO POGUANTO Anak dari P...
5,754/Pid.Sus/2024/PN Tng,PN TANGERANG,1 (satu) paket plastik klip narkotika jenis sh...,Menyatakan Terdakwa RAMDHANI BIN. DJAMALUDIN M...
...,...,...,...,...
83,467/Pid.Sus/2024/PN Tng,PN TANGERANG,5 (lima) bungkus plastik klip bening didalamny...,Menyatakan Terdakwa Dimas Dwi Febytrandi alias...
84,Pdt G PA Pal,PA PALU,Mut?ah sejumlah Rp10.000.000.00 (sepuluh juta ...,DALAM KONVENSI Mengabulkan permohonan Pemohon ...
86,502/Pid.Sus/2024/PN Tng,PN TANGERANG,1 (satu) bungkus klip berisi 3 (tiga) bungkus ...,Menyatakan Terdakwa I RIYAN MULYA bin MUZIZAT ...
87,Pdt G PA Pal,PA PALU,Mut?ah sejumlah Rp10.000.000.00 (sepuluh juta ...,DALAM KONVENSI Mengabulkan permohonan Pemohon ...


In [ ]:
dataset_evaluasi

,Nomor,Lembaga Peradilan,Barang Bukti,Catatan Amar
91,431/Pid.Sus/2024/PN Tng,PN TANGERANG,26 (dua puluh enam) bungkus plastik klip (kode...,Menyatakan Terdakwa Muhamad Fahri Ramadan alia...
92,Pdt G PA Pal,PA PALU,Mut?ah sejumlah Rp10.000.000.00 (sepuluh juta ...,DALAM KONVENSI Mengabulkan permohonan Pemohon ...
93,Pdt G PA Pal,PA PALU,Mut?ah sejumlah Rp10.000.000.00 (sepuluh juta ...,DALAM KONVENSI Mengabulkan permohonan Pemohon ...
95,554/Pid.Sus/2024/PN Tng,PN TANGERANG,1 (satu) buah paket J&T Express Courier resi J...,Menyatakan terdakwa MOHAMMAD RIZKI FADHILA Ali...
97,194/Pid.Sus/2024/PN Tng,PN TANGERANG,1 (satu) buah bekas bungkus rokok Merek On Bol...,Menyatakan Terdakwa Abdul Rosyid Alias Ocid Bi...
...,...,...,...,...
323,2100/Pid.Sus/2023/PN Tng,PN TANGERANG,1 (satu) buah tas warna biru yang didalamnya t...,Menyatakan Terdakwa JANRI HERY HALATOKAN als B...
324,1984/Pid.Sus/2023/PN Tng,PN TANGERANG,1 (satu) bungkus plastic bening berisikan Nark...,Menyatakan Terdakwa Asep Hidayat Als Tekel Bin...
325,2023/Pid.Sus/2023/PN Tng,PN TANGERANG,2 (dua) buah Plastik bening yang diduga berisi...,Menyatakan Terdakwa I Mochamad Rizky Waluyo Al...
326,1906/Pid.Sus/2023/PN Tng,PN TANGERANG,2 (dua) bungkus plastik bening yang didalamnya...,Menyatakan terdakwa ABDULLOH SOLEHUDIN alias B...


#Langkah 2: Menghasilkan Dataset Pertanyaan-Jawaban
Gunakan model Vicuna untuk menghasilkan jawaban dari dataset.

In [ ]:
def setup_vicuna(model_name="TheBloke/vicuna-7B-1.1-HF"):
    os.makedirs("./offload", exist_ok=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name, legacy=False)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        offload_folder="./offload"
    )
    model.generation_config.pad_token_id = tokenizer.eos_token_id
    print("Model Vicuna berhasil dimuat.")
    return tokenizer, model

def generate_answer(question, context, tokenizer, model):
    input_text = f"Context: {context}\nQuestion: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(inputs.input_ids, max_new_tokens=50)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.split("Answer:")[-1].strip()

def generate_dataset(dataset):
    tokenizer, model = setup_vicuna()
    questions_per_case = [
        "Apa barang bukti dalam kasus ini?",
        "Apa keputusan hakim?",
        "Siapa terdakwa yang terlibat dalam kasus ini?",
        "Apa yang menjadi pertimbangan hakim dalam menjatuhkan denda?"
    ]
    results = []
    for _, row in dataset.iterrows():
        context = (
            f"No Putusan: {row['Nomor']}. "
            f"Barang Bukti: {row['Barang Bukti']}. "
            f"Amar Putusan: {row['Catatan Amar']}."
        )
        for question in questions_per_case:
            answer = generate_answer(question, context, tokenizer, model)
            results.append({"No Putusan": row["Nomor"], "Pertanyaan": question, "Jawaban": answer})
    return pd.DataFrame(results)

# Generate dataset pertanyaan-jawaban
qa_dataset = generate_dataset(dataset_model)
qa_dataset.to_csv("qa_dataset.csv", index=False)
print("Dataset pertanyaan-jawaban berhasil disimpan.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `pad_token_id` should be positive but got -1. This will cause errors when batch generating, if there is padding. Please set `pad_token_id` explicitly as `model.generation_config.pad_token_id=PAD_TOKEN_ID` to avoid errors in generation
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Model Vicuna berhasil dimuat.


OutOfMemoryError: CUDA out of memory. Tried to allocate 202.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 33.06 MiB is free. Process 255708 has 14.71 GiB memory in use. Of the allocated memory 14.14 GiB is allocated by PyTorch, and 461.04 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

#Langkah 3: Embeddings dan Pencocokan
Menggunakan SentenceTransformer untuk embeddings dan mencocokkan pertanyaan baru.

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

qa_df = pd.read_csv('qa_dataset.csv')
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedding_model.encode(qa_df['Pertanyaan'].tolist())
np.save('embeddings.npy', embeddings)

def find_relevant_answer(new_question):
    new_embedding = embedding_model.encode([new_question])
    scores = cosine_similarity(new_embedding, embeddings)
    best_match_idx = scores.argmax()
    return qa_df.iloc[best_match_idx]

# Contoh pertanyaan baru
new_question = "Apa jenis narkotika yang ditemukan dalam kasus ini?"
relevant_qa = find_relevant_answer(new_question)
print("Pertanyaan yang paling relevan:", relevant_qa['Pertanyaan'])
print("Jawaban:", relevant_qa['Jawaban'])


#Langkah 4: Evaluasi Model
Evaluasi hasil model berdasarkan dataset-evaluasi.

In [ ]:
from sklearn.metrics import classification_report

def evaluate_model_on_dataset(dataset, ground_truth_col, predicted_col):
    dataset['Evaluation'] = dataset[ground_truth_col] == dataset[predicted_col]
    dataset['Evaluation'] = dataset['Evaluation'].map({True: "Benar", False: "Salah"})
    print("Classification Report:\n")
    print(classification_report(dataset['Evaluation'], dataset['Evaluation']))

# Tambahkan kolom Ground Truth (simulasi jawaban benar)
qa_df['Ground Truth'] = qa_df['Pertanyaan'].apply(lambda x: "Jawaban yang benar untuk " + x)
evaluate_model_on_dataset(qa_df, ground_truth_col='Ground Truth', predicted_col='Jawaban')
